<a href="https://colab.research.google.com/github/1NourHany/Alzheimer-s-Disease-Detection-/blob/main/final_vgg_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task: Improve Accuracy

The task is to improve the accuracy from 90% to 98-99%. Densenet161 gives 90% accuracy. The task is to optimize for accuracy but I will also show the F Beta score of the model as well. This notebook will feature a vgg16 model, trained in fastAI, and using progressive resizing and cutout to attain better results. Updated for september, I added some additional visuals to help understand the training and added a new section at the end for practical applications in healthcare.

The statement below was from the original 'completed' version of this notebook about 5 months ago. I am thrilled to say that this is no longer the case as an academic team improved upon my work to achieve truly phonomenal results
`
The model only manages to achieve 95% accuracy, so I would consider this model a large step in the right direction, but not quite reaching the destination. Feel free to fork this notebook and play around, maybe you will find the missing link to reach the 98%+ mark. Check my final thoughts section for ideas for further improvement
`
The current best performing model is now getting 99% on a custom test set, I highly recommend checking it out in the link to the paper in the special thanks section.

Special Thanks to:
* Sarvesh Dubey for both the dataset and the Task https://www.kaggle.com/tourist55
* Zachary Burns, Derrick Cosmas, and Bryce Smith for taking the project I started here and improving upon it (I think its my first time being cited in an acedemic paper) http://noiselab.ucsd.edu/ECE228/projects/Report/52Report.pdf

<a id="TOC"></a>
## Table of Contents

* [Part 2: Pint sized model](#part-two) 
* [part 3: Full sized fun](#part-three) 
    * [Potholes in high dimensional space](#pihds) 
* [Part 4: Test sets in FastAI](#part-four) 
    * [What does the model see](#wdtms) 
* [Conclusions](#Conclusions) 


# Part one: feeding in the data

In [ ]:
!pip install torch==1.4.0

In [ ]:
!pip install torchvision==0.5.0

In [ ]:
!pip install http://download.pytorch.org/whl/cpu/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip install fastai==1.0.60

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

In [ ]:
import zipfile
local_zip = '/content/alzheimers-dataset-4-class-of-images.zip'
data = zipfile.ZipFile(local_zip, 'r')
data.extractall('/content/Alzheimer Date')
data.close()

In [ ]:
import numpy as np
import pandas as pd
import os
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [ ]:
PATH = Path('/content/Alzheimer Date/Alzheimer_s Dataset/')


In [ ]:
transform = get_transforms(max_rotate=7.5,
                           max_zoom=1.15,
                           max_lighting=0.15,
                           max_warp=0.15,
                           p_affine=0.8, p_lighting = 0.8, 
                           xtra_tfms= [
                               pad(mode='zeros'),
                               symmetric_warp(magnitude=(-0.1,0.1)),
                               cutout(n_holes=(1,3), length=(5,5))
                           ])

In [ ]:
data = ImageDataBunch.from_folder(PATH, train="train/",
                                  test="test/",
                                  valid_pct=.4,
                                  ds_tfms=transform,
                                  size=112,bs=64, 
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(10,10))

In [ ]:
Category.__eq__ = lambda self, that: self.data == that.data
Category.__hash__ = lambda self: hash(self.obj)
Counter(data.train_ds.y)

In [ ]:
import torch.nn as nn

learn = cnn_learner(data, models.vgg16_bn, metrics=[FBeta(average='weighted'),accuracy], wd=1e-1, callback_fns=ShowGraph)

#learn = cnn_learner(data, models.vgg16_bn,loss_func=catagoricalcrossentropy , metrics=error_rate, wd=1e-1)#,pretrained=False)
learn.fit_one_cycle(4)

In [ ]:
Model_Path = Path('/content/drive/MyDrive/vgg')
learn.model_dir = Model_Path
learn.save('checkpoint-1')

In [ ]:
learn.model
learn.recorder.plot_losses()

In [ ]:
learn.load('checkpoint-1');

In [ ]:
learn.unfreeze()
#learn.fit_one_cycle(8, max_lr=slice(1e-6,3e-4))
learn.fit_one_cycle(4, max_lr=slice(1e-6,3e-4))

In [ ]:
learn.model
learn.recorder.plot_losses()

In [ ]:
learn.save('checkpoint-2')

This project uses FastAI version 1.0.60, the new version have several syntax differences and new features.

### Cutout
Cutout is a method for removing small chunks of an image at random (see the batch images below to get an idea). This was one of the transforms that I experimented the most with and it helped close the gap a little between validation accuracy and test accuracy. 

I explored many different models and found VGG16 to have more consistent performance. Other similar performing models were resnet50, resnet101, resnet152, densenet161 and vgg19_bn. I experimented with squeezenet but the performance was awful. I had also tried googlenet and inception v3, but these methods I was unable to make compatible with progressive resizing. 

I'm relatively new to DeepLearning with FastAI, from my limited experience with Progressive Resizing, the smaller training phases are most useful for the final product when they are still a bit underfit to the dataset. Further experimentation with this could, and possibly should be done to confirm.
<a id="part-three"></a>
## Part 3: Full sized fun!

Now that there is a reasonably good model for 112x112 images, now I will train the model on full sized images. Since I am now creating a final model, I will include the F Beta score to get a more complete view of the models performance. I also will modify the cutout section of the transforms to upscale the cutout sizes. 

[Back to table of contents](#TOC) 

In [ ]:
learn.destroy()


In [ ]:
transform = get_transforms(max_rotate=7.5,
                           max_zoom=1.15,
                           max_lighting=0.15,
                           max_warp=0.15,
                           p_affine=0.8,
                           p_lighting = 0.8,
                           xtra_tfms= [
                               pad(mode='zeros'),
                               symmetric_warp(magnitude=(-0.1,0.1)),
                               cutout(n_holes=(1,6),length=(5,20))])

In [ ]:
data = ImageDataBunch.from_folder(PATH, train="train/",
                                 valid="train/",
                                  test="test/",
                                  valid_pct=.2,
                                  ds_tfms=transform,
                                  size=224,bs=32, 
                                  ).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.vgg16_bn, pretrained=False, metrics=[error_rate, FBeta(average='weighted'),accuracy], wd=1e-1, callback_fns=ShowGraph)
Model_Path = Path('/content/drive/MyDrive/vgg')
learn.model_dir = Model_Path
learn.load('checkpoint-2');


In [ ]:
learn.lr_find()
learn.recorder.plot()

<a id="pihds"></a>
# Potholes in high dimensional space:

If you look at the training epochs below you will notice that the model hit a series of local minima on the way towards the optimal weights. I experimented with many combinations of weight decays, training cycles, learning rates, fine tuning, and the combination below worked the best. Even with these parameters the model still fumbles and falls on its face a little as it tries to find the optimal point.You can get an idea of how bunpy a ride it is with the added visual below.

Smaller learning rates kept getting stuck in local minima then overfitting, higher learning rates kept overshooting and getting worse without overfitting. Fine tuning the model by unfreezing it and retraining the early layers with a mild learning rate caused the error rate to go through the roof. Weight decay worked just fine at 1e-1, did some experimenting but the accuracy never got much higher than 80% with other weight decay settings. As a funny side note I tested this with multiple different models, and the learning rate curves optimal point stayed the same, but the steepness of the slope was different (the larger models had a flatter curve while the smaller models had a sharper one). 

In [ ]:
learn.fit_one_cycle(17, max_lr=5e-4)

In [ ]:
learn.save('checkpoint-3')

In [ ]:
learn.model
learn.recorder.plot_losses()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(8,8))

<a id="part-four"></a>
# Part 4: Test sets in FastAI

FastAI's data bunch doesn't attempt to compare test data with any labels, it assumes that the test set is only there to be labelled. As such, the cell below allows me to test the dataset on the test set and view the results. The results are output in an n+1 size list, where n refers to the number of evaluation metrics the model is given. I'm not sure what the first entry in the list does, each of the following entries are the scores of the model, output in the order in which they are received.

[Back to table of contents](#TOC) 

In [ ]:
transform = get_transforms()

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
random_seed(42, True)

In [ ]:
data_test =  ImageDataBunch.from_folder(PATH,
                                        #ignore_empty=True,
                                  train="test/",#"train/",
                                  valid="test/",
                                  valid_pct=.95,
#                                  ds_tfms=transform,
                                  size=224,bs=32,
                                  num_workers=0
                                  ).normalize(imagenet_stats)
ev = learn.validate(data_test.train_dl,metrics=[error_rate, FBeta(average='weighted'),accuracy])
print('Results from test set \tError rate:', float(ev[1]), '\tF Beta Score: ', float(ev[2]),'Accuracy rate:', float(ev[3]))

<a id="wdtms"></a>
### What does the model see?
Below is where the real meat of this notebook is. We have a function which shows what areas of the image contributed the most to the appropriate diagnostic. With this method, one can view an image, see what the algorithmn predicted and why. This can help humans involved in the diagnostic process get a second opinion or bring attention high risk areas.

As a side note, I find it interesting that for alzheimers free and early stage MRI images the areas that are most highlighted are around the ventricals (the tubes in the center) while the late stage images often have the outer regions highlighted (which appear to be less dense).

In [ ]:
# The code below is slighty modified from https://www.kaggle.com/daisukelab/verifying-cnn-models-with-cam-and-etc-fast-ai
# and that code was a hevily modified version of https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson6-pets-more.ipynb

from fastai.callbacks.hooks import *

def visualize_cnn_by_cam(learn, data_index):
    x, _y = learn.data.valid_ds[data_index]
    y = _y.data
    if not isinstance(y, (list, np.ndarray)): # single label -> one hot encoding
        y = np.eye(learn.data.valid_ds.c)[y]

    m = learn.model.eval()
    xb,_ = learn.data.one_item(x)
    xb_im = Image(learn.data.denorm(xb)[0])
    xb = xb.cuda()

    def hooked_backward(cat):
        with hook_output(m[0]) as hook_a: 
            with hook_output(m[0], grad=True) as hook_g:
                preds = m(xb)
                preds[0,int(cat)].backward()
        return hook_a,hook_g
    def show_heatmap(img, hm, label):
        _,axs = plt.subplots(1, 2)
        axs[0].set_title(label)
        img.show(axs[0])
        axs[1].set_title(label)
        img.show(axs[1])
        axs[1].imshow(hm, alpha=0.6, extent=(0,img.shape[1],img.shape[1],0),
                      interpolation='bilinear', cmap='magma');
        plt.show()

    for y_i in np.where(y > 0)[0]:
        hook_a,hook_g = hooked_backward(cat=y_i)
        acts = hook_a.stored[0].cpu()
        grad = hook_g.stored[0][0].cpu()
        grad_chan = grad.mean(1).mean(1)
        mult = (acts*grad_chan[...,None,None]).mean(0)
        show_heatmap(img=xb_im, hm=mult, label=str(learn.data.valid_ds.y[data_index]))


idx_list = [0,1,2,31,3,63, 142, 207]        
#idx_list = range(200,220)
for idx in idx_list:# range(10):
    visualize_cnn_by_cam(learn, idx)

<a id="conclusions"></a>

 # Final Thoughts:

The goal of the Task was to hit an accuracy of 98% or greater, looks like the approach I chose to take just gets me to around 95% accuracy with an f_beta score of around .95 (Didn't set any random states so those number can be off by a percentage point or two when you rerun the notebook). The one nitpick about the model I have is that it will generate a small number of false negatives (the model would predict someone does not have alzheimers when they do).

In training and validation the model is able to get an accuracy score as high as 99% and an F_beta of around .98, however on test data the model is only able to achieve 95%. The final piece in getting this model's performance up is to focus on how to tone the overfitting down a smidge (In my personal experience, the best performing models are always slightly overfit). There is probably room for improvement with the transforms that I chose for the images. There may also be room for improvement by adding another resizing step or changing some of the dimensions in the current step. It's also possible that combining the results from this model with another would be necessary to close the gap for 98%.

In 3/21/2020, this was the most accurate model for this dataset that I'm aware of. Now this model isn't the most accurate, but the heatmap interptiability adds a lot of value to this project

This year I've challenged myself to complete one task on Kaggle per week, in order to develop a larger Data Science portfolio. If you found this notebook useful or interesting please give it an upvote. I'm always open to constructive feedback. If you have any questions, comments, concerns, or if you would like to collaborate on a future task of the week feel free to leave a comment here or message me directly. For past TOTW check out the link to my page on github for this ongoing project
https://github.com/Neil-Kloper/Weekly-Kaggle-Task/wiki

[Back to table of contents](#TOC) 